# Setup

**GPU:** Use either a Nvidia T4 GPU or an Nvidia K80 GPU and verify which GPU.


**SE Types:** Reconnaissance, Social Engineering, Malware, Phishing


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Wed Apr  7 17:08:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-P

**download_gpt2: only run ONCE to download**

**file upload from local machine: only upload once into files**

In [ ]:
import os
from google.colab import files

os.chdir('/content')
uploaded = files.upload()

Saving fraudulent_emails_v2.json to fraudulent_emails_v2.json


In [ ]:
import json
messages = json.load(open('fraudulent_emails_v2.json',))

recon_train = []
se_train = []
phish_train = []
mal_train = []

for message in messages:
    if 'se_tag' in message:
        ttr_dict = message['TTR_text']
        t = ttr_dict['ttr_text']

        if t != '':
            if message['se_tag'] == 'recon':
                recon_train.append(t)

            if message['se_tag'] == 'social_engineering':
                se_train.append(t)

            if message['se_tag'] == 'phishing':
                phish_train.append(t)

            elif message['se_tag'] == 'malware':
                mal_train.append(t)

            else:
                continue
    else:
        print('hi')
    

print("Recon train:", len(recon_train))
print("SE messages:", len(se_train))
print("Phishing messages: ", len(phish_train))
print("Malware messages: ", len(mal_train))

print(446+1706+1447+79)


Recon train: 446
SE messages: 1706
Phishing messages:  1447
Malware messages:  79
3678


In [ ]:
# write out text files with messages by attacker type
# use to generate later
with open("recon_train.txt", "w") as output:
    output.write(str(recon_train))

with open("se_train.txt", "w") as output:
    output.write(str(se_train))

with open("phish_train.txt", "w") as output:
    output.write(str(phish_train))

with open("mal_train.txt", "w") as output:
    output.write(str(mal_train)) 

In [ ]:
import re

tokens =[]
for r in recon_train:
    res = len(re.findall(r'\s+', r))
    tokens.append(res)

recon_avg_length = int(sum(tokens)/len(tokens))

tokens =[]
for se in se_train:
    result = len(re.findall(r'\s+', se))
    tokens.append(result)

se_avg_length = int(sum(tokens)/len(tokens))

tokens =[]
for ph in phish_train:
    result = len(re.findall(r'\s+', ph))
    tokens.append(result)

ph_avg_length = int(sum(tokens)/len(tokens))

tokens =[]
for mal in mal_train:
    result = len(re.findall(r'\s+', mal))
    tokens.append(result)

mal_avg_length = int(sum(tokens)/len(tokens))

print(mal_avg_length)
print(se_avg_length)
print(recon_avg_length)
print(ph_avg_length)

473
426
456
475


# Restart Runtime Cells

In [ ]:
%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Wed Apr  7 18:45:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-P

# Finetune GPT2 Model

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 333Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.56Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 260Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:17, 28.7Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 185Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.74Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 2.92Mit/s]


In [ ]:
recon_gen_file = 'gpt2_gentext_recon_{:%Y%m%d}.txt'.format(datetime.utcnow())
se_gen_file = 'gpt2_gentext_se_{:%Y%m%d}.txt'.format(datetime.utcnow())
phish_gen_file = 'gpt2_gentext_phish_{:%Y%m%d}.txt'.format(datetime.utcnow())
mal_gen_file = 'gpt2_gentext_mal_{:%Y%m%d}.txt'.format(datetime.utcnow())

## Recon Model


In [ ]:
file_name = "recon_train.txt"

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=500,
              restore_from='fresh',
              run_name='recon',
              print_every=20,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.17s/it]


dataset has 322965 tokens
Training...
[20 | 33.67] loss=2.96 avg=2.96
[40 | 58.90] loss=2.19 avg=2.57
[60 | 84.14] loss=2.35 avg=2.50
[80 | 109.39] loss=2.17 avg=2.41
[100 | 134.58] loss=2.12 avg=2.35
======== SAMPLE 1 ========
 drafting to me at the expiration of 2 years, which means that I have until June 14th, 1999 to get out of this fund. On this date, I shall notify you in writing of my intentions and intentions. Moreso, I desire you to be a person of interest to me because i will not be able to work in the finance. Please reply back to my e-mail address.\nI wait in anticipation of your best regards,\nMr. Kenneth Zadakovsky.\r\nRegards,\r\nMrs. Eleni Tish.\r\nYours faithfully,\r \nMs. Atsu Kova .", 'FROM MR PETRES\nDear Friend \nI am MR PETRES, my Manager at Billing/Repatriation Account, my main purpose in contacting you was to make a proposal to you for the \ndeal of your money, to be paid into your bank account through our company, because it is a \nreliable and reputable organi

In [ ]:
gpt2.generate_to_file(sess, destination_path = recon_gen_file, model_name='124M',                      
                      length = recon_avg_length,
                      temperature = 0.8, nsamples = 200, batch_size = 20)

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Wed Apr  7 18:11:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-P

## Social Engineering Model


In [ ]:
# se model finetune
file_name = "se_train.txt"

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=500,
              restore_from='fresh',
              run_name='se',
              print_every=20,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:06<00:00,  6.99s/it]


dataset has 1197813 tokens
Training...
[20 | 31.55] loss=2.93 avg=2.93
[40 | 56.72] loss=2.75 avg=2.84
[60 | 81.87] loss=2.51 avg=2.73
[80 | 107.02] loss=2.44 avg=2.65
[100 | 132.16] loss=2.39 avg=2.60
======== SAMPLE 1 ========
 took you?\r\nMolly?\r\nmoney\r\nI have some\r\nmoney to give you.\r\nnope, sorry\r\nbut\r', "URGENT!\nBEST\r\nMRS SHIRDEGAR.\r\n\r\n\r\nTHANK YOU.\r\nI AM MRS SHIRDEGAR.\r\nI AM AFRICA SENIAM KAIKURA\r\nJOHANNESBURG.\r\nTHANK YOU,\r\nTHE MANAGED DESK OF\r\nMRS SHIRDEGAR LAGADO.\r\nMEMBED\r\nI am Mary Sagadero, widow to Sir Hula Sagadero, president of the Senegal Association of Football (SAFA).\r\nwah this is what I do as husband and wife,we don\r\nlive for. I\r\ndecided over the retirement of some members of OUR SENEGAL ASSOCIATION,because they have\r\nthe desire to retire in order to invest in\r\nor help them (I have also decided to apply for a position in various foreign\r\ninvestments, so I would appreciate if you consider helping me\r\ndetermine where and 

In [ ]:
gpt2.generate_to_file(sess, destination_path = se_gen_file, model_name='124M',                      
                      length = se_avg_length,
                      temperature = 0.8, nsamples = 200, batch_size = 20)

## Phishing Model


In [ ]:
file_name = "phish_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=500,
              restore_from='fresh',
              run_name='phish',
              print_every=20,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:06<00:00,  6.51s/it]


dataset has 1154205 tokens
Training...
[20 | 31.63] loss=2.80 avg=2.80
[40 | 56.80] loss=2.39 avg=2.59
[60 | 81.94] loss=2.72 avg=2.64
[80 | 107.14] loss=2.31 avg=2.55
[100 | 132.32] loss=2.29 avg=2.50
======== SAMPLE 1 ========
 Joseph is here for the fun. I hope you will treat me quite good.I will be expecting you to contact me in my private email address, at this address and the phone number that you provided.\r\nPlease do consider this a very big deal to you, I can't wait to hear from you as soon as possible. \r\nBest regards,\r\nDr. Alex Toussi.\r', "FROM THE DESK OF MICHAEL JONES WHO CARRYED URGENTLY FOUR HUNDRED AND TWENTY DOLLARS IN OUR COUNTRY.\nDear Sir,\nPlease.\r\nI AM SUSAN JONES,\r\nTHE CONFIDENTIAL OFFICE DESK OF MR. ROBERT MOHM. I HAVE DUTY DISCOVERED MY HUSBAND AT THE SAME TIME IN MY CUSTOMER\r\nDEAR BOY,\r\nTHE DIRECTOR OF MY CAR\r\nDEAR HUSBAND IS HAVING AN OVER-EXAMINED DUTY, WHICH\r\nHAVE BEEN TOLD TO GET INTO THE TRACING COOPERATION WITH THE TRADING AND\r\nCUSTOME

In [ ]:
gpt2.generate_to_file(sess, destination_path = phish_gen_file, model_name='124M',
                      length = ph_avg_length, 
                      temperature = 0.8, nsamples = 200, batch_size = 20)

## Malware Model


In [ ]:
# malware model finetune
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset="mal_train.txt",
              model_name='124M',
              steps=500,
              restore_from='fresh',
              run_name='mal',
              print_every=25,
              sample_every=200,
              save_every=50
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.76it/s]


dataset has 64312 tokens
Training...
[25 | 38.74] loss=2.38 avg=2.38
[50 | 70.20] loss=1.07 avg=1.72
Saving checkpoint/mal/model-50
[75 | 104.92] loss=1.15 avg=1.53
[100 | 136.37] loss=0.81 avg=1.34
Saving checkpoint/mal/model-100
Instructions for updating:
Use standard file APIs to delete files with this prefix.
[125 | 170.28] loss=0.92 avg=1.26
[150 | 201.72] loss=0.36 avg=1.10
Saving checkpoint/mal/model-150
[175 | 235.90] loss=0.18 avg=0.97
[200 | 267.36] loss=0.05 avg=0.85
Saving checkpoint/mal/model-200
======== SAMPLE 1 ========
 not\nknowingly or intentionally sending funds or goods or services to any bank or other financial institution in the euro area. 2006\nThe Financial Conduct\nsystem shall handle this transaction with utmost confidentiality but note \nthat your identity or the location of the payment\nservice can not be determined until after the completion\nto which the institution presents the appropriate documents. \nAfter which, the institution shall deal with you ide

In [ ]:
mal_gen_file = 'gpt2_gentext_mal_{:%Y%m%d}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess, destination_path = mal_gen_file, model_name='124M',                      
                      length = mal_avg_length,
                      temperature = 0.8, nsamples = 200, batch_size = 20)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

gpt2.copy_checkpoint_to_gdrive(run_name='mal')
files.download(mal_gen_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Save to Drive

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='recon')
gpt2.copy_checkpoint_to_gdrive(run_name='se')
gpt2.copy_checkpoint_to_gdrive(run_name='phish')
#gpt2.copy_checkpoint_to_gdrive(run_name='mal')

# Generate Text From Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
# may have to run twice to get file to download
files.download(recon_gen_file)
files.download(se_gen_file)
files.download(phish_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='mal')
files.download(mal_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
recon_gen_file = 'gpt2_gentext_recon_{:%Y%m%d}.txt'.format(datetime.utcnow())
se_gen_file = 'gpt2_gentext_se_{:%Y%m%d}.txt'.format(datetime.utcnow())
phish_gen_file = 'gpt2_gentext_phish_{:%Y%m%d}.txt'.format(datetime.utcnow())
mal_gen_file = 'gpt2_gentext_mal_{:%Y%m%d}.txt'.format(datetime.utcnow())

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

# Force-kill VM

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Load Trained Model Checkpoint

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

##  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: February 14th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

# Unused Code

In [ ]:
gpt2.generate(sess, run_name='run1')

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.